In [54]:
import matplotlib.pyplot as plt
import math
import numpy as np
import threading

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

In [19]:
# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Gate, Instruction, circuit, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice, AwsQuantumTask
from braket.ir.openqasm import Program as OpenQASMProgram

# Please enter the S3 bucket you created during onboarding in the code below
my_bucket = f"amazon-braket-Your-Bucket-Name" # the name of the bucket
my_prefix = "Your-Folder-Name" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

backend_rigetti = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1")
supported_gates = backend_rigetti.properties.action['braket.ir.jaqcd.program'].supportedOperations
print('Gate set supported by the Rigetti backend device:\n', supported_gates, '\n')

backend_ionq = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
supported_gates = backend_ionq.properties.action['braket.ir.jaqcd.program'].supportedOperations
print('Gate set supported by the IonQ backend device:\n', supported_gates, '\n')

Gate set supported by the Rigetti backend device:
 ['cz', 'xy', 'ccnot', 'cnot', 'cphaseshift', 'cphaseshift00', 'cphaseshift01', 'cphaseshift10', 'cswap', 'h', 'i', 'iswap', 'phaseshift', 'pswap', 'rx', 'ry', 'rz', 's', 'si', 'swap', 't', 'ti', 'x', 'y', 'z', 'start_verbatim_box', 'end_verbatim_box'] 

Gate set supported by the IonQ backend device:
 ['x', 'y', 'z', 'rx', 'ry', 'rz', 'h', 'cnot', 's', 'si', 't', 'ti', 'v', 'vi', 'xx', 'yy', 'zz', 'swap', 'i'] 



# Generate a random bitstring using a quantum circuit

## Basic Hadamard transform

In theory, a basic Hadamard transform would be sufficient to generate a true, unbiased random bitstring.

In [16]:
def hadamard_transform(n_qubits):
    """
    function to apply Hadamard gate on each qubit
    input: number of qubits
    """

    # instantiate circuit object
    circuit = Circuit()

    # apply series of Hadamard gates
    for i in range(n_qubits):
        circuit.h(i)

    return circuit

In [26]:
n_qubits = 80
circ = hadamard_circuit(n_qubits)
print(circ)

T   : |0|
         
q0  : -H-
         
q1  : -H-
         
q2  : -H-
         
q3  : -H-
         
q4  : -H-
         
q5  : -H-
         
q6  : -H-
         
q7  : -H-
         
q8  : -H-
         
q9  : -H-
         
q10 : -H-
         
q11 : -H-
         
q12 : -H-
         
q13 : -H-
         
q14 : -H-
         
q15 : -H-
         
q16 : -H-
         
q17 : -H-
         
q18 : -H-
         
q19 : -H-
         
q20 : -H-
         
q21 : -H-
         
q22 : -H-
         
q23 : -H-
         
q24 : -H-
         
q25 : -H-
         
q26 : -H-
         
q27 : -H-
         
q28 : -H-
         
q29 : -H-
         
q30 : -H-
         
q31 : -H-
         
q32 : -H-
         
q33 : -H-
         
q34 : -H-
         
q35 : -H-
         
q36 : -H-
         
q37 : -H-
         
q38 : -H-
         
q39 : -H-
         
q40 : -H-
         
q41 : -H-
         
q42 : -H-
         
q43 : -H-
         
q44 : -H-
         
q45 : -H-
         
q46 : -H-
         
q47 : -H-
         
q48 : -H-
         


Let's measure this on Rigetti and see the results.

In [28]:
job = backend_rigetti.run(circ, s3_folder, shots=10, poll_timeout_seconds=5*24*60*60)

print('Job ID:', job.id)
print('Status of task:', job.state())

counts = job.result().measurement_counts
print(f'\nRandom bitstring: {list(counts.keys())[0]}')

Job ID: arn:aws:braket:us-west-1:592242689881:quantum-task/c6b81e53-ece7-41c7-9d10-61c2501d924b
Status of task: QUEUED

Random bitstring: 10000000000000000000001000000001001000100100010001010000100100011000000010000000


The problem with a simple Hadamard transform is that the results will never be truly unbiased on noisy quantum hardware. This is due to many factors, but namely systematic unitary errors from imperfectly calibrated gates, unwanted couplings, and crosstalk between qubits. To generate a more unabiased random number, we need to try to remove this bias using random circuit sampling, similar to Google's quantum supremacy experiments.

# Random circuit sampling

## Functions

In [ ]:
def rand_all_to_all_circ(n_qubits, n_layers, seed=None):
    """A function to prepare an all-to-all circuit with random SU(2) gates"""
    
    if seed is not None:
        np.random.seed(seed)
    def single_random_layers(n_qubits, depth):
        def gen_layer():
            for q in range(n_qubits):
                angle = np.random.uniform(0, 2 * math.pi)
                gate = np.random.choice([Gate.Rx(angle), Gate.Ry(angle), Gate.Rz(angle)], 1, replace=True)[0]
                yield (gate, q)
        for _ in range(depth):
            yield gen_layer()
    
    circ = Circuit()
    circs_single = single_random_layers(n_qubits, n_layers+1)

    for layer in range(n_layers):
        for sq_gates in next(circs_single):
            gate, target = sq_gates
            circ.add_instruction(Instruction(gate, target))

        # match the qubits into pairs
        x = np.arange(n_qubits)
        np.random.shuffle(x)
        for i in range(0, n_qubits - 1, 2):
            i, j = x[i], x[i + 1]
            circ.cnot(i, j)

    # last layer of single qubit rotations
    for sq_gates in next(circs_single):
        gate, target = sq_gates
        circ.add_instruction(Instruction(gate, target))

    return circ

In [43]:
def run_circuit():

    circ = all_to_all(11, 30)
    
    # Only measure each circuit once
    job = backend_ionq.run(circ, s3_folder, shots=1, poll_timeout_seconds=5*24*60*60)
    id_array.append(job.id)

    print('Job ID:', job.id)
    print('Status of job:', job.state(), '\n')

    counts = job.result().measurement_counts
    random_bitstrings.append(list(counts.keys())[0])

## Measure

Here is an example circuit with random SU(2) gates and randomly-sampled CNOT gates. This type of circuit is best implemented on an all-to-all QPU. Therefore, we implement this on IonQ's hardware.

In [30]:
circ = all_to_all(11, 3)
print(circ)

T   : |   0    |      1       |   2    |     3      |   4    |        5         |   6    |
                                                                                          
q0  : -Rx(4.64)------------C---Rx(2.19)---C----------Ry(3.83)---X----------------Ry(0.34)-
                           |              |                     |                         
q1  : -Rx(4.20)------------|-C-Ry(5.68)---|-X--------Rx(3.09)---|-----X----------Ry(6.18)-
                           | |            | |                   |     |                   
q2  : -Ry(4.30)-Ry(0.93)---|-|----------C-|-|--------Rx(1.47)-X-|-----|----------Rx(0.23)-
                           | |          | | |                 | |     |                   
q3  : -Rx(1.53)-C----------|-|-Ry(1.71)-X-|-|--------Rx(3.72)-|-|---C-|----------Rz(2.60)-
                |          | |            | |                 | |   | |                   
q4  : -Rz(1.21)-|--------C-|-|-Rz(2.04)---|-C--------Rx(2.01)-|-|-C-|-|----------Ry(5.25)-

For the actual experiment, we generate a much deeper circuit 10x longer than the example circuit above. We desire a bitstring that is 6 x 64 = 384 bits because it encodes 6 base 64 characters in a user friendly manner. Since IonQ has 11 qubits, this requires measuring 35 different random circuits, concatenating their bitstrings, and then truncating it at a length of 384.

In [44]:
bitstring_length = 64 * 6
n_qubits = 11
n_circuits = int(np.ceil(bitstring_length / n_qubits))

id_array = []
random_bitstrings = []
for i in range(n_circuits):
    thread = threading.Thread(target=run_circuit, args=())
    thread.setDaemon(True)
    thread.start()
    print(f"Thread {i} started")

Thread 0 started
Thread 1 started
Thread 2 started
Thread 3 started
Thread 4 started
Thread 5 started
Thread 6 started
Thread 7 started
Thread 8 started
Thread 9 started
Thread 10 started
Thread 11 started
Thread 12 started
Thread 13 started
Thread 14 started
Thread 15 started
Thread 16 started
Thread 17 started
Thread 18 started
Thread 19 started
Thread 20 started
Thread 21 started
Thread 22 started
Thread 23 started
Thread 24 started
Thread 25 started
Thread 26 started
Thread 27 started
Thread 28 started
Thread 29 started
Thread 30 started
Thread 31 started
Thread 32 started
Thread 33 started
Thread 34 started


[]

Job ID: arn:aws:braket:us-east-1:592242689881:quantum-task/05614783-99ae-41d2-95c2-46f8b21e4498
Job ID: arn:aws:braket:us-east-1:592242689881:quantum-task/e30b0746-f0d3-4e7f-85b5-1a44c5655d4f
Job ID: arn:aws:braket:us-east-1:592242689881:quantum-task/83cddf7a-a6fb-4fb5-986d-5001fce89d02
Job ID: arn:aws:braket:us-east-1:592242689881:quantum-task/5a44466a-44b8-4b59-b43c-834cb291681b
Job ID: arn:aws:braket:us-east-1:592242689881:quantum-task/e1bac7c3-3cc1-455b-aea7-bdbfbd4a23e6
Job ID: arn:aws:braket:us-east-1:592242689881:quantum-task/8dc09f3f-0cdc-4388-a6b6-1ab6ca90b5fa
Job ID: arn:aws:braket:us-east-1:592242689881:quantum-task/cf2b3446-aaf3-427e-b2e0-05fe3dd6583d
Job ID: arn:aws:braket:us-east-1:592242689881:quantum-task/de3ef775-1ef7-446f-a999-3fc0d9e55010
Job ID: arn:aws:braket:us-east-1:592242689881:quantum-task/c03dbba0-4eef-4808-9441-d28094505f80
Job ID: arn:aws:braket:us-east-1:592242689881:quantum-task/98780973-583a-492b-8df8-ce143a6955ce
Job ID: arn:aws:braket:us-east-1:5922426

Check the status of our jobs:

In [59]:
def get_task_status(i):

    task_load = AwsQuantumTask(arn=i)

    # print status
    status = task_load.state()
    print('Status of (reconstructed) task:', status, '\n')
    
for i in id_array:
    thread = threading.Thread(target=get_task_status, args=(str(i),))
    thread.setDaemon(True)
    thread.start()

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: COMPLETED 

Status of (reconstructed) task: 

In [56]:
id_array

['arn:aws:braket:us-east-1:592242689881:quantum-task/05614783-99ae-41d2-95c2-46f8b21e4498',
 'arn:aws:braket:us-east-1:592242689881:quantum-task/e30b0746-f0d3-4e7f-85b5-1a44c5655d4f',
 'arn:aws:braket:us-east-1:592242689881:quantum-task/83cddf7a-a6fb-4fb5-986d-5001fce89d02',
 'arn:aws:braket:us-east-1:592242689881:quantum-task/5a44466a-44b8-4b59-b43c-834cb291681b',
 'arn:aws:braket:us-east-1:592242689881:quantum-task/e1bac7c3-3cc1-455b-aea7-bdbfbd4a23e6',
 'arn:aws:braket:us-east-1:592242689881:quantum-task/8dc09f3f-0cdc-4388-a6b6-1ab6ca90b5fa',
 'arn:aws:braket:us-east-1:592242689881:quantum-task/cf2b3446-aaf3-427e-b2e0-05fe3dd6583d',
 'arn:aws:braket:us-east-1:592242689881:quantum-task/de3ef775-1ef7-446f-a999-3fc0d9e55010',
 'arn:aws:braket:us-east-1:592242689881:quantum-task/c03dbba0-4eef-4808-9441-d28094505f80',
 'arn:aws:braket:us-east-1:592242689881:quantum-task/98780973-583a-492b-8df8-ce143a6955ce',
 'arn:aws:braket:us-east-1:592242689881:quantum-task/ae7060b6-8e5d-4e08-80a7-690

Here are all of the random bitstrings that we measured:

In [61]:
random_bitstrings

['01100011111',
 '10010011111',
 '10011111010',
 '01100110101',
 '11011001001',
 '11000011100',
 '10000000110',
 '11011101010',
 '01010000001',
 '10110010001',
 '11000000011',
 '10011000100',
 '00001000101',
 '10110011111',
 '11000110000',
 '01100101011',
 '10100001111',
 '11111110111',
 '01000100101',
 '00101011001',
 '10000000101',
 '11001101011',
 '01001001101',
 '10101100000',
 '11110101001',
 '10101111110',
 '11100001110',
 '11111001111',
 '01101000111',
 '00101100101',
 '00110101011',
 '00110101010',
 '01001100110',
 '10010001101']

Now let's concatenate them and truncate the final random bitstring of length 384.

In [60]:
bitstring = ''.join(random_bitstrings)[:bitstring_length]
bitstring

'01100011111100100111111001111101001100110101110110010011100001110010000000110110111010100101000000110110010001110000000111001100010000001000101101100111111100011000001100101011101000011111111111011101000100101001010110011000000010111001101011010010011011010110000011110101001101011111101110000111011111001111011010001110010110010100110101011001101010100100110011010010001101'

So, the central question is, why is this a better way of generating a random number on noisy quantum hardware than a simple Hadamard transform? The answer is two-fold: firstly, we generate our random circuits using Haar-random single-qubit gates. This means that we are uniformly randomly sampling them from SU(2), so each random circuit itself will be impacted by a different systematic bias. This is the same idea that is implemented in randomized benchmarking experiments: by uniformly sampling from the Clifford group, randomized benchmarking transforms all systematic errors into depolarizing noise to produce an average error rate. So, we take this same idea and implement it randomly sampled circuits to produce random outputs. The second reason why this idea is better than a Hadamard transform is that Google's quantum supremacy experiments [Nature volume 574, pages 505–510 (2019)] demonstrated that random circuits of this form cannot be efficiently simulated using classical computers. For example, they showed that their largest random circuits would take approximately 10,000 years to simulate classically. Therefore, our random number generator not only implements a more unbiased random bitstring, but it is not efficient to simulate classically, making it more robust for the purposes of cybersecurity.